# Initialization

In [ ]:
#@markdown #Mount Google Drive
#@markdown - To save time and avoid repeatedly uploading a large audio dataset, the RAVDESS audio dataset can be stored in a Google Drive and loaded directly from there.

#@markdown - By storing the dataset in Google Drive, it can be easily accessed and loaded into Colab. This method is much faster than uploading the dataset each time Colab is opened. Additionally, it ensures that the dataset is always available, and the user does not need to worry about losing or misplacing the files.

#@markdown - Overall, storing large datasets in Google Drive is an efficient and convenient way to manage data, especially for machine learning projects that require large amounts of data.

mount_drive = True #@param {type:"boolean"}
if mount_drive:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import re
import numpy as np

In [ ]:
#@markdown # Dataset Directory
dataset_path = "/content/drive/MyDrive/Datasets/Bangla Audio/RAVDESS" #@param {type:"string"}

file_list = os.listdir(dataset_path)
print(f"Audio Files: {file_list}")
print(f"Total Audio Files: {len(file_list)}")

Audio Files: ['03-01-07-01-02-02-08.wav', '03-01-03-02-01-01-08.wav', '03-01-04-02-02-02-08.wav', '03-01-06-02-01-02-08.wav', '03-01-04-01-01-01-08.wav', '03-01-04-02-01-01-08.wav', '03-01-02-02-01-01-08.wav', '03-01-07-01-01-01-08.wav', '03-01-05-01-02-02-08.wav', '03-01-03-02-02-02-08.wav', '03-01-05-02-01-01-08.wav', '03-01-08-02-02-02-07.wav', '03-01-04-01-02-02-08.wav', '03-01-05-02-02-02-08.wav', '03-01-05-01-02-01-08.wav', '03-01-04-02-02-01-08.wav', '03-01-03-02-02-01-08.wav', '03-01-01-01-02-02-08.wav', '03-01-02-02-02-02-08.wav', '03-01-02-01-01-02-08.wav', '03-01-06-02-01-01-08.wav', '03-01-07-01-01-02-08.wav', '03-01-02-02-02-01-08.wav', '03-01-02-01-01-01-08.wav', '03-01-03-01-02-02-08.wav', '03-01-06-01-02-01-08.wav', '03-01-03-01-02-01-08.wav', '03-01-06-02-02-01-08.wav', '03-01-02-01-01-02-09.wav', '03-01-04-02-02-02-09.wav', '03-01-05-02-02-01-09.wav', '03-01-06-01-02-02-09.wav', '03-01-06-01-01-01-09.wav', '03-01-07-02-02-01-08.wav', '03-01-07-02-01-01-08.wav', '03-01

In [ ]:
#@markdown # Create a Dataframe of the Dataset

gender_list = []
speaker_id_list = []
speaker_name_list = []
sentence_id_list = []
emotion_list = []
take_id_list = []

for file_name in file_list:
    # Using regular expressions to extract the information from the file name
    parts = file_name.split("-")

    gender = "F" if int(parts[6][:-4]) % 2 == 0 else "M"
    actor_number = parts[6][:-4]
    statement_number = parts[4]
    emotion = {
        "01": "NEUTRAL",
        "02": "CALM",
        "03": "HAPPY",
        "04": "SAD",
        "05": "ANGRY",
        "06": "FEARFUL",
        "07": "DISGUST",
        "08": "SURPRISED"
        }[parts[2]]

    repetition = parts[5]

    # Append the extracted information to the respective lists
    gender_list.append(gender)
    speaker_name_list.append(actor_number)
    sentence_id_list.append(statement_number)
    emotion_list.append(emotion)
    take_id_list.append(repetition)

# Creating a dictionary with the lists as values
data = {
    'gender': gender_list,
    'speaker_name': speaker_name_list,
    'sentence_id': sentence_id_list,
    'emotion': emotion_list,
    'take_id': take_id_list
}

# Creating the DataFrame
df = pd.DataFrame(data)
print("Dataframe created and the variable is df.")


Dataframe created and the variable is df.


In [ ]:
def tags(file_name):
  parts = file_name.split("-")
  gender = "F" if int(parts[6][:-4]) % 2 == 0 else "M"
  actor_number = parts[6][:-4]
  statement_number = parts[4]
  emotion = {
      "01": "NEUTRAL",
      "02": "CALM",
      "03": "HAPPY",
      "04": "SAD",
      "05": "ANGRY",
      "06": "FEARFUL",
      "07": "DISGUST",
      "08": "SURPRISED"
      }[parts[2]]

  repetition = parts[5]

  return {"gender": gender, "speaker": actor_number, "emotion": emotion}


In [ ]:
#@markdown # Install Libraries

update_apt = True #@param {type:"boolean"}
install_pyaudio = False #@param {type:"boolean"}
install_librosa = False #@param {type:"boolean"}
install_soundfile = False #@param {type:"boolean"}
install_resampy = False #@param {type:"boolean"}

if install_librosa:
  !pip install librosa

if install_soundfile:
  !pip install soundfile

if update_apt:
  !apt-get update

if install_pyaudio:
  !apt-get install -y python3-pyaudio

if install_resampy:
  !pip install resampy


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


# Importing the required libraries

In [ ]:
import librosa
import soundfile
import glob
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import joblib
import time
import datetime

# Extracting features from the audio files
Extracting importand data that can be captured from audio files.

- **MFCCs (Mel-Frequency Cepstral Coefficients):** These coefficients represent the spectral envelope of a signal and capture information about the shape of the vocal tract. They are commonly used in speech and music analysis.

- **Chroma:** Chroma features map the distribution of pitches in a musical signal to a 12-dimensional vector, with each element representing the energy in a particular pitch class.

- **Mel-scaled spectrogram:** This representation of the power spectral density of a signal transforms the frequency axis to a perceptually based scale called the mel scale.

- **Spectral centroid:** This feature measures the average frequency of a signal weighted by its spectral magnitude and is a measure of the center of gravity of the spectrum.

- **Spectral flux:** This feature measures the rate of change of the spectral magnitude over time and is a measure of the local spectral dynamics of the signal.

- **Spectral rolloff:** This feature is a measure of the bandwidth of the signal and represents the frequency below which a certain percentage of the total spectral energy lies.

- **Zero-crossing rate:** This feature measures the rate at which the signal changes from positive to negative or vice versa and is a measure of the temporal dynamics of the signal.

In [ ]:
def extract_features(file_name, mfcc, chroma, mel, spec_centroid, spec_flux, spec_rolloff, zero_crossing):
    X, sample_rate = librosa.load(os.path.join(file_name), res_type="kaiser_fast")

    if chroma:
        stft = np.abs(librosa.stft(X))
    result = np.array([])

    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
        result = np.hstack((result, mel))
    if spec_centroid:
        centroid = np.mean(librosa.feature.spectral_centroid(y=X, sr=sample_rate).T, axis=0)
        result = np.hstack((result, centroid))
    if spec_flux:
        flux = np.mean(librosa.onset.onset_strength(y=X, sr=sample_rate).T, axis=0)
        result = np.hstack((result, flux))
    if spec_rolloff:
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=X, sr=sample_rate).T, axis=0)
        result = np.hstack((result, rolloff))
    if zero_crossing:
        zcr = np.mean(librosa.feature.zero_crossing_rate(y=X).T, axis=0)
        result = np.hstack((result, zcr))
    return result

<div align="center">

<br>
<br>
<br>
<br>

# Emotion Recognition

<br>
<br>
<br>
<br>


</div>


# Loading the data for feature extraction from the audio files

In [ ]:
def load_data(mfcc, chroma, mel, spec_centroid, spec_flux, spec_rolloff, zero_crossing, test_size=0.2, path="/content/drive/MyDrive/Datasets/Bangla Audio/RAVDESS"):
  x, y = [], []

  for file in os.listdir(dataset_path):
    file_name = file

    # Getting the emotion label
    # match = re.match(r'[MF]_\d{2}_[A-Z]+_S_\d+_(\w+)_\d+\.wav', file_name)
    emotion = tags(file_name)["emotion"]
    feature = extract_features(f"/content/drive/MyDrive/Datasets/Bangla Audio/RAVDESS/{file}", mfcc=mfcc, chroma=chroma, mel=mel,
                               spec_centroid=spec_centroid, spec_flux=spec_flux, spec_rolloff=spec_rolloff, zero_crossing=zero_crossing)
    x.append(feature)
    y.append(emotion)

  return train_test_split(np.array(x), y, test_size=test_size, train_size=0.75, random_state=9)


# Train Test Split

In [ ]:
#@markdown # Extract Features from Audio Files and Train Test Split for **Emotion Recognition**
#@markdown ---

directory = "/content/drive/MyDrive/Datasets/Bangla Audio/Serialized/"

#@markdown ### Run Train Test Split and Save the Data
run_train_test_split_for_emotion_clf = True #@param {type:"boolean"}
#@markdown ---

#@markdown ### Select Audio Features

mfcc = True #@param {type:"boolean"}
chroma = True #@param {type:"boolean"}
mel = True #@param {type:"boolean"}
spec_centroid = True #@param {type:"boolean"}
spec_flux = True #@param {type:"boolean"}
spec_rolloff = True #@param {type:"boolean"}
zero_crossing = True #@param {type:"boolean"}



#@markdown If "run_train_test_split_for_emotion_clf" is True, a serialized object will be stored in "/content/drive/MyDrive/Datasets/Bangla Audio/Serialized/". Give a file name to store these values "x_train, x_test, y_train, y_test".

train_test_split_for_emotion_file_name = "all_possible_features_ravdess_train_test_split_for_emotion_1" #@param {type:"string"}
directory += f"{train_test_split_for_emotion_file_name}.joblib"


if run_train_test_split_for_emotion_clf:
  start_time = time.time()

  x_train, x_test, y_train, y_test = load_data(mfcc, chroma, mel,
                                               spec_centroid, spec_flux,
                                               spec_rolloff, zero_crossing, test_size=0.25)

  end_time = time.time()

  total_time = end_time - start_time
  print("Total time taken to extract features from the audio files:", total_time, "seconds")

  joblib.dump((x_train, x_test, y_train, y_test), directory)
  print(f"Train Test Split for Emotion CLF is serialized and saved in this directory: {directory}")



#@markdown ---
#@markdown <br>
#@markdown <br>
#@markdown <center><b>Or</b></center>
#@markdown <br>
#@markdown <br>

#@markdown ---
#@markdown ## Load Existing Train Test Split Data
#@markdown When I did my research on emotion classification, I performed a train-test split on the dataset. Later on, I saved the split data using joblib so that I can import it for later use instead of re-performing the split every time. This approach saves time and computational resources.



load_emotion_test_split = False #@param {type:"boolean"}



#@markdown Train Test Split for Emotion will be loaded and saved in 'x_train, x_test, y_train, y_test'.
if load_emotion_test_split:
  print("Locating previously saved train test split data for emotion clf.")
  existing_train_test_split_emotion = "all_possible_features_ravdess_train_test_split_for_emotion_1" #@param {type:"string"}
  existing_train_test_split_emotion = f"/content/drive/MyDrive/Datasets/Bangla Audio/Serialized/{existing_train_test_split_emotion}.joblib"
  print(f"Found a latest data in {existing_train_test_split_emotion}")

  x_train, x_test, y_train, y_test = joblib.load(existing_train_test_split_emotion)
  print("Train Test Split for Emotion is loaded and saved successfully in 'x_train, x_test, y_train, y_test'.")

#@markdown ---

print(f"\nShapes: {x_train.shape[0], x_test.shape[0]}")
print(f"Extracted Features: {x_train.shape[1]}")
print(f"\nx_train: {x_train} \n\ny_train: {y_train[:10]} First 10 values of y_train.")

Total time taken to extract features from the audio files: 292.289715051651 seconds
Train Test Split for Emotion CLF is serialized and saved in this directory: /content/drive/MyDrive/Datasets/Bangla Audio/Serialized/all_possible_features_ravdess_train_test_split_for_emotion_1.joblib

Shapes: (1080, 360)
Extracted Features: 184

x_train: [[-5.86849854e+02  5.64196129e+01  2.17711639e+00 ...  1.32276762e+00
   5.02394798e+03  1.45755124e-01]
 [-6.01510681e+02  6.52067795e+01  1.47881413e+01 ...  1.27828014e+00
   4.82980266e+03  1.73407405e-01]
 [-6.09250488e+02  6.50298233e+01  2.08593512e+00 ...  1.31471038e+00
   4.85980152e+03  1.53126293e-01]
 ...
 [-6.30160645e+02  4.32521439e+01  3.66772223e+00 ...  1.24539101e+00
   5.87036472e+03  1.98639927e-01]
 [-4.28872925e+02  1.39648409e+01 -3.55706329e+01 ...  1.41080475e+00
   5.64118652e+03  2.48854034e-01]
 [-4.63667694e+02  2.33761749e+01 -3.53788643e+01 ...  1.47587132e+00
   5.05508662e+03  2.37298622e-01]] 

y_train: ['SAD', 'DISGU

<div align="center">

<br>
<br>
<br>
<br>

# Gender Recognition

<br>
<br>
<br>
<br>


</div>


# Loading the data for feature extraction from the audio files

In [ ]:
def load_data(mfcc, chroma, mel, spec_centroid, spec_flux, spec_rolloff, zero_crossing, test_size=0.2, path="/content/drive/MyDrive/Datasets/Bangla Audio/RAVDESS"):
  x, y = [], []

  for file in os.listdir(dataset_path):
    file_name = file

    # Getting the gender label
    # match = re.match(r'([MF])_\d{2}_[A-Z]+_S_\d+_(\w+)_\d+\.wav', file_name)
    emotion = tags(file_name)["gender"]
    feature = extract_features(f"/content/drive/MyDrive/Datasets/Bangla Audio/RAVDESS/{file}", mfcc=mfcc, chroma=chroma, mel=mel,
                               spec_centroid=spec_centroid, spec_flux=spec_flux, spec_rolloff=spec_rolloff, zero_crossing=zero_crossing)
    x.append(feature)
    y.append(emotion)

  return train_test_split(np.array(x), y, test_size=test_size, train_size=0.75, random_state=9)


In [ ]:
gender_classes = ["Female", "Male"]

# Train Test Split

In [ ]:
#@markdown # Extract Features from Audio Files and Train Test Split for **Gender Recognition**
#@markdown ---

#@markdown ## Run Train Test Split and Save the Data
run_train_test_split_for_gender_clf = True #@param {type:"boolean"}
directory = "/content/drive/MyDrive/Datasets/Bangla Audio/Serialized/"




#@markdown ### Select Audio Features

mfcc = True #@param {type:"boolean"}
chroma = True #@param {type:"boolean"}
mel = True #@param {type:"boolean"}
spec_centroid = True #@param {type:"boolean"}
spec_flux = True #@param {type:"boolean"}
spec_rolloff = True #@param {type:"boolean"}
zero_crossing = True #@param {type:"boolean"}




#@markdown If "run_train_test_split_for_gender_clf" is True, a serialized object will be stored in "/content/drive/MyDrive/Datasets/Bangla Audio/Serialized/". Give a file name to store these values "x_train, x_test, y_train, y_test".

train_test_split_for_gender_file_name = "all_possible_features_ravdess_train_test_split_for_gender_1" #@param {type:"string"}
directory += f"{train_test_split_for_gender_file_name}.joblib"


if run_train_test_split_for_gender_clf:
  start_time = time.time()
  x_train, x_test, y_train, y_test = load_data(mfcc, chroma, mel,
                                               spec_centroid, spec_flux,
                                               spec_rolloff, zero_crossing,test_size=0.25)
  end_time = time.time()

  total_time = end_time - start_time
  print("Total time taken to extract features from the audio files:", total_time, "seconds")

  joblib.dump((x_train, x_test, y_train, y_test), directory)
  print(f"Train Test Split for Gender CLF is serialized and saved in this directory: {directory}")




#@markdown ---
#@markdown <br>
#@markdown <br>
#@markdown <center><b>Or</b></center>
#@markdown <br>
#@markdown <br>

#@markdown ---
#@markdown ## Load Existing Train Test Split Data
#@markdown When I did my research on gender classification, I performed a train-test split on the dataset. Later on, I saved the split data using joblib so that I can import it for later use instead of re-performing the split every time. This approach saves time and computational resources.




load_gender_test_split = False #@param {type:"boolean"}

#@markdown Train Test Split for Gender will be loaded and saved in 'x_train, x_test, y_train, y_test'.
if load_gender_test_split:
  print("Locating previously saved train test split data for gender clf.")
  existing_train_test_split_gender = "all_possible_features_ravdess_train_test_split_for_gender_1" #@param {type:"string"}
  existing_train_test_split_gender = f"/content/drive/MyDrive/Datasets/Bangla Audio/Serialized/{existing_train_test_split_gender}.joblib"
  print(f"Found a latest data in {existing_train_test_split_gender}")

  x_train, x_test, y_train, y_test = joblib.load(existing_train_test_split_gender)
  print("Train Test Split for Gender is loaded and saved successfully in 'x_train, x_test, y_train, y_test'.")

#@markdown ---


print(f"\nShapes: {x_train.shape[0], x_test.shape[0]}")
print(f"Extracted Features: {x_train.shape[1]}")
print(f"\nx_train: {x_train} \n\ny_train: {y_train[:10]} First 10 values of y_train.")

Total time taken to extract features from the audio files: 288.4270849227905 seconds
Train Test Split for Gender CLF is serialized and saved in this directory: /content/drive/MyDrive/Datasets/Bangla Audio/Serialized/all_possible_features_ravdess_train_test_split_for_gender_1.joblib

Shapes: (1080, 360)
Extracted Features: 184

x_train: [[-5.86849854e+02  5.64196129e+01  2.17711639e+00 ...  1.32276762e+00
   5.02394798e+03  1.45755124e-01]
 [-6.01510681e+02  6.52067795e+01  1.47881413e+01 ...  1.27828014e+00
   4.82980266e+03  1.73407405e-01]
 [-6.09250488e+02  6.50298233e+01  2.08593512e+00 ...  1.31471038e+00
   4.85980152e+03  1.53126293e-01]
 ...
 [-6.30160645e+02  4.32521439e+01  3.66772223e+00 ...  1.24539101e+00
   5.87036472e+03  1.98639927e-01]
 [-4.28872925e+02  1.39648409e+01 -3.55706329e+01 ...  1.41080475e+00
   5.64118652e+03  2.48854034e-01]
 [-4.63667694e+02  2.33761749e+01 -3.53788643e+01 ...  1.47587132e+00
   5.05508662e+03  2.37298622e-01]] 

y_train: ['F', 'M', 'M',

<div align="center">

<br>
<br>
<br>
<br>

# Speaker Recognition

<br>
<br>
<br>
<br>


</div>


# Loading the data for feature extraction from the audio files

In [ ]:
def load_data(mfcc, chroma, mel, spec_centroid, spec_flux, spec_rolloff, zero_crossing, test_size=0.2, path="/content/drive/MyDrive/Datasets/Bangla Audio/RAVDESS"):
  x, y = [], []

  for file in os.listdir(dataset_path):
    file_name = file

    # Getting the speaker label
    # match = re.match(r'[MF]_\d{2}_(\w+)+_S_\d+_(\w+)_\d+\.wav', file_name)
    emotion = tags(file_name)["speaker"]
    feature = extract_features(f"/content/drive/MyDrive/Datasets/Bangla Audio/RAVDESS/{file}", mfcc=mfcc, chroma=chroma, mel=mel,
                               spec_centroid=spec_centroid, spec_flux=spec_flux, spec_rolloff=spec_rolloff, zero_crossing=zero_crossing)
    x.append(feature)
    y.append(emotion)

  return train_test_split(np.array(x), y, test_size=test_size, train_size=0.75, random_state=9)


# Train Test Split

In [ ]:
#@markdown # Extract Features from Audio Files and Train Test Split for **Speaker Recognition**
#@markdown ---

#@markdown ## Run Train Test Split and Save the Data
run_train_test_split_for_speaker_clf = True #@param {type:"boolean"}
directory = "/content/drive/MyDrive/Datasets/Bangla Audio/Serialized/"




#@markdown ### Select Audio Features

mfcc = True #@param {type:"boolean"}
chroma = True #@param {type:"boolean"}
mel = True #@param {type:"boolean"}
spec_centroid = True #@param {type:"boolean"}
spec_flux = True #@param {type:"boolean"}
spec_rolloff = True #@param {type:"boolean"}
zero_crossing = True #@param {type:"boolean"}




#@markdown If "run_train_test_split_for_speaker_clf" is True, a serialized object will be stored in "/content/drive/MyDrive/Datasets/Bangla Audio/Serialized/". Give a file name to store these values "x_train, x_test, y_train, y_test".

train_test_split_for_speaker_file_name = "all_possible_features_ravdess_train_test_split_for_speaker_1" #@param {type:"string"}
directory += f"{train_test_split_for_speaker_file_name}.joblib"


if run_train_test_split_for_speaker_clf:
  start_time = time.time()
  x_train, x_test, y_train, y_test = load_data(mfcc=mfcc, chroma=chroma, mel=mel,
                                               spec_centroid=spec_centroid, spec_flux=spec_flux,
                                               spec_rolloff=spec_rolloff, zero_crossing=zero_crossing, test_size=0.25)
  end_time = time.time()

  total_time = end_time - start_time
  print("Total time taken to extract features from the audio files:", total_time, "seconds")

  joblib.dump((x_train, x_test, y_train, y_test), directory)
  print(f"Train Test Split for Speaker CLF is serialized and saved in this directory: {directory}")

#@markdown ---
#@markdown <br>
#@markdown <br>
#@markdown <center><b>Or</b></center>
#@markdown <br>
#@markdown <br>

#@markdown ---
#@markdown ## Load Existing Train Test Split Data
#@markdown When I did my research on speaker classification, I performed a train-test split on the dataset. Later on, I saved the split data using joblib so that I can import it for later use instead of re-performing the split every time. This approach saves time and computational resources.

load_speaker_test_split = False #@param {type:"boolean"}

#@markdown Train Test Split for Speaker will be loaded and saved in 'x_train, x_test, y_train, y_test'.
if load_speaker_test_split:
  print("Locating previously saved train test split data for speaker clf.")
  existing_train_test_split_speaker = "all_possible_features_ravdess_train_test_split_for_speaker_1" #@param {type:"string"}
  existing_train_test_split_speaker = f"/content/drive/MyDrive/Datasets/Bangla Audio/Serialized/{existing_train_test_split_speaker}.joblib"
  print(f"Found a latest data in {existing_train_test_split_speaker}")

  x_train, x_test, y_train, y_test = joblib.load(existing_train_test_split_speaker)
  print("Train Test Split for Speaker is loaded and saved successfully in 'x_train, x_test, y_train, y_test'.")

#@markdown ---


print(f"\nShapes: {x_train.shape[0], x_test.shape[0]}")
print(f"Extracted Features: {x_train.shape[1]}")
print(f"\nx_train: {x_train} \n\ny_train: {y_train[:10]} First 10 values of y_train.")

Total time taken to extract features from the audio files: 289.5120735168457 seconds
Train Test Split for Speaker CLF is serialized and saved in this directory: /content/drive/MyDrive/Datasets/Bangla Audio/Serialized/all_possible_features_ravdess_train_test_split_for_speaker_1.joblib

Shapes: (1080, 360)
Extracted Features: 184

x_train: [[-5.86849854e+02  5.64196129e+01  2.17711639e+00 ...  1.32276762e+00
   5.02394798e+03  1.45755124e-01]
 [-6.01510681e+02  6.52067795e+01  1.47881413e+01 ...  1.27828014e+00
   4.82980266e+03  1.73407405e-01]
 [-6.09250488e+02  6.50298233e+01  2.08593512e+00 ...  1.31471038e+00
   4.85980152e+03  1.53126293e-01]
 ...
 [-6.30160645e+02  4.32521439e+01  3.66772223e+00 ...  1.24539101e+00
   5.87036472e+03  1.98639927e-01]
 [-4.28872925e+02  1.39648409e+01 -3.55706329e+01 ...  1.41080475e+00
   5.64118652e+03  2.48854034e-01]
 [-4.63667694e+02  2.33761749e+01 -3.53788643e+01 ...  1.47587132e+00
   5.05508662e+03  2.37298622e-01]] 

y_train: ['10', '11', 